In [4]:
# Install Qdrant client library
!pip install qdrant-client


  Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
Using cached protobuf-5.29.1-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.10.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 5.29.1 which is incompatible.
google-cloud-bigquery-storage 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.1 which is incompatible

In [5]:
!docker run -p 6333:6333 -d qdrant/qdrant


6854fcf7f0d86c7a4cdff7695b4c8fed8a027dc321f21b40f6ec115e86b71d78


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import pandas as pd
from qdrant_client.models import PointStruct
import os


## Connect to Qdrant

In [7]:
# Initialize the Qdrant client
client = QdrantClient(host="127.0.0.1", port=6333)

# Check server connection
try:
    response = client.get_collections()
    print("Qdrant is running. Collections:", response)
except Exception as e:
    print(f"Error connecting to Qdrant: {e}")


Qdrant is running. Collections: collections=[]


## Process Data Using Hugging Face Transformers

In [8]:
from transformers import AutoTokenizer, AutoModel

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()


2024-12-17 14:39:55.996052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# df = pd.read_csv("/Users/afnanalamri/Desktop/MyProject/LeapAI/WebCrawlingTask/raw_data/filtered_contractors_data.csv")

# Specify the relative path to the data file
current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
data_path = os.path.join(current_dir, "raw_data", "filtered_contractors_data.csv")

# Read file using relative path
df = pd.read_csv(data_path)

# print("Data loaded successfully!")
# print(df.head())



df['Combined_Text'] = df[['Company Name', 'Company Size', 'Interests', 'City', 'Address']].fillna('').agg(' '.join, axis=1)

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

df['embedding'] = df['Combined_Text'].apply(generate_embedding)

# # Save the updated DataFrame
# df.to_csv("/Users/afnanalamri/Desktop/MyProject/LeapAI/WebCrawlingTask/raw_data/contractors_with_embeddings.csv", index=False)

# Save the updated DataFrame using a relative path
output_path = os.path.join(current_dir, "raw_data", "contractors_with_embeddings.csv")
df.to_csv(output_path, index=False)


In [10]:
df.head()

,Company Name,Membership Number,Company Size,Organization Email,Address,City,Region,Interests,Combined_Text,embedding
0,Gulf Pioneers Trading Company,280728074,Medium Company Size,Acc@Gpksa.Com,ALtahlia Street\r\nOlaya\r\nRiyadh\r\n54355\r\...,RIYADH,Riyadh,"Construction of buildings, Construction of bui...",Gulf Pioneers Trading Company Medium Company S...,"[0.04954048991203308, 0.03295160084962845, 0.0..."
1,Ratel Al Sharq Contracting Company,101010104,Small Company Size,Adel_77@Hotmail.Com,Riyadh - Al Narjis District - Anas Bin Malik Road,RIYADH,Riyadh,"Construction of buildings, Construction of bui...",Ratel Al Sharq Contracting Company Small Compa...,"[0.0501224510371685, 0.037430986762046814, -0...."
2,Bunoon Wa Funoon Contracting Co.,138113818,Small Company Size,Info@Bfconst.Com,"6394, Al Olaya\r\n3891, Al Olaya Dist.\r\nRIYA...",RIYADH,Riyadh,"Construction of buildings, Construction of bui...",Bunoon Wa Funoon Contracting Co. Small Company...,"[0.12054764479398727, 0.0346849262714386, 0.08..."
3,Al-Wessam Contracting Company,106110611,Small Company Size,Alwessam7@Gmail.Com,Al Wessam Contracting Company,BILJURASHI,Bahah,"Mining support services, Oil and natural gas e...",Al-Wessam Contracting Company Small Company Si...,"[-0.02744017168879509, -0.03267991170287132, 0..."
4,TAKWEEN AND CONSTRUCTION,438043804,Very Small Company Size,B2B@Tbco.Com.Sa,Not Available,RIYADH,Riyadh,"Construction of buildings, Construction of bui...",TAKWEEN AND CONSTRUCTION Very Small Company Si...,"[0.04318187013268471, -0.059580229222774506, 0..."


## Store data in Qdrant

In [11]:
# Create a new collection in Qdrant
client.recreate_collection(
    collection_name="contractors",
    vectors_config=VectorParams(
        size=384,
        distance=Distance.COSINE
    )
)

#Preparing points to upload to Qdrant
points = [
    PointStruct(
        id=i,
        vector=row['embedding'],
        payload={
            "Company Name": row["Company Name"],
            "Company Size": row["Company Size"],
            "Interests": row["Interests"],
            "City": row["City"],
            "Address": row["Address"],
        }
    )
    for i, row in df.iterrows()
]

# Upload points to Qdrant
client.upsert(collection_name="contractors", points=points)

print("Embeddings uploaded successfully!")


/var/folders/8j/m_1dsv_x5dlf53pndg9t48680000gn/T/ipykernel_18442/3271033850.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Embeddings uploaded successfully!


In [12]:
print(type(df['embedding'].iloc[0]))


<class 'list'>
